## OpenAQ data preparation

In [2]:
pip install ee

Processing c:\users\ayako\appdata\local\pip\cache\wheels\0b\12\c0\b52afbb4093f67c02dac5869ef6bad845a234e2bd76bd803e9\ee-0.2-py3-none-any.whl
  Using cached blessings-1.7-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install windows-curses

  Using cached windows_curses-2.2.0-cp38-cp38-win_amd64.whl (88 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install google-api-python-client

  Using cached uritemplate-3.0.1-py2.py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.  Downloading google_api_core-1.26.3-py2.py3-none-any.whl (93 kB)

  Using cached cachetools-4.2.1-py3-none-any.whl (12 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)


In [5]:
pip install earthengine-api

  Using cached google_api_python_client-1.12.8-py2.py3-none-any.whl (61 kB)
Processing c:\users\ayako\appdata\local\pip\cache\wheels\04\5d\b9\a746c136fae5811cb2d30d8112b0f72ba40e24d22885bbe7c7\httplib2shim-0.0.3-py2.py3-none-any.whl
  Using cached google_cloud_core-1.6.0-py2.py3-none-any.whl (28 kB)
  Using cached google_resumable_media-1.2.0-py2.py3-none-any.whl (75 kB)
  Using cached google_crc32c-1.1.2-cp38-cp38-win_amd64.whl (34 kB)
  Created wheel for earthengine-api: filename=earthengine_api-0.1.259-py3-none-any.whl size=178440 sha256=1681a616786113ce50032cca5df4cff638e8e6b75281314ac556035d19c589ec
  Stored in directory: c:\users\ayako\appdata\local\pip\cache\wheels\80\f5\85\7114145ab1527a91247f9b9e65b20516905a2ae44fb35fe9c3
Successfully built earthengine-api
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.1.0
    Uninstalling google-api-python-client-2.1.0:
      Successfully uninstalled google-api-python-client-2.1.0


In [7]:
import pandas as pd
import ee
from tqdm import tqdm # Progress bar
from datetime import timedelta

In [8]:
ee.Authenticate()  # Run once to link your Google Earth Engine account

Enter verification code: 4/1AY0e-g6mEyWvsyk7GuZeMv4vyLQoddowd8RYAmXo4JVl1QXIVdLcFDgkD4g

Successfully saved authorization token.


In [9]:
ee.Initialize() # Run each session

In [10]:
world = pd.read_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/02_OpenAQ/world_openaq.csv')
world.sample(5)

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
2260301,SAB-RAME - Sabaneta - I.E. Rafael J. Mejía,pm25,9.53084,b'\xc2\xb5g/m\xc2\xb3',CO,Sabaneta,2020-06-22 13:00:00+00:00,6.145500,-75.621262
2111363,CAL-JOAR - Caldas - E U Joaquín Aristizabal,pm25,22.76360,b'\xc2\xb5g/m\xc2\xb3',CO,Caldas,2020-10-17 11:00:00+00:00,6.093078,-75.637764
741005,"Jadavpur, Kolkata - WBPCB",pm25,69.70000,b'\xc2\xb5g/m\xc2\xb3',IN,Kolkata,2020-03-18 09:15:00+00:00,22.499290,88.369170
1479551,Karachi,pm25,25.30000,b'\xc2\xb5g/m\xc2\xb3',PK,NaN,2020-01-28 14:00:00+00:00,24.841500,67.009100
2101201,US Diplomatic Post: Bogota,pm25,16.00000,b'\xc2\xb5g/m\xc2\xb3',CO,Bogota,2020-11-16 20:00:00+00:00,4.637735,-74.094860


In [11]:
world = world.rename(columns={"date.utc": "Date", "coordinates.latitude": "Lat", "coordinates.longitude":"Long", "value": "PM25"})
world['Date'] = world['Date'].astype(str).str[:10]
world = world.drop(columns=['unit', 'parameter'], axis =1)
world = world[['location', 'Date', 'city', 'country', 'PM25', 'Lat','Long']]
world['PM25'] = world.groupby(['Date','location'])['PM25'].transform(lambda x: x.mean())
world = world.groupby(['Date', 'location']).first().reset_index()

In [12]:
world.shape

(96195, 7)

## Getting Data from Google Earth Engine

### Population density, night light data

In [13]:
# A utility function to pull data for a set of locations
def sample(im, prop, lats, lons, scale=100, reducer=ee.Reducer.first(), tileScale=4):
    points = []
    for lat, lon in zip(lats, lons):
        xy = ee.Geometry.Point([lon, lat])
        points.append(xy.buffer(scale))
    vals = im.reduceRegions(collection=ee.FeatureCollection(points), scale=scale, reducer=reducer, tileScale=tileScale).getInfo()
    if prop == '':
        return [v['properties'] for v in vals['features']]
    return [v['properties'][prop] for v in vals['features']]

# Example:
lights = ee.ImageCollection("NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4") # An image collection
lights = lights.filter(ee.Filter.date('2010-01-01', '2018-03-08')).first() # An image
vals = sample(lights, '', [-18.136], [30.15], scale=100) # See image value for a single location
vals

[{'avg_vis': 8.628110885620117, 'cf_cvg': 84}]

In [14]:
#population data
def add_static_vars(df, scale = 2000):
    lights = ee.ImageCollection("NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4").filter(ee.Filter.date('2010-01-01', '2018-03-08')).first()
    pop = ee.ImageCollection("CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density").filter(ee.Filter.date('2010-01-01', '2018-03-08')).first()
    ims = [lights, pop]

    for im in tqdm(ims):
        for i, reducer in enumerate([ee.Reducer.mean(), ee.Reducer.min(), ee.Reducer.max()]):
            sampled_values = sample(im, '', df['Lat'].values, df['Long'].values, reducer=reducer)
            for k in sampled_values[0].keys():
                arr = ['mean', 'min', 'max']
                df[k+'_'+ arr[i]] = [sv[k] if k in sv.keys() else None for sv in sampled_values]
                if k == arr[i]:
                    df = df.rename(columns={k+'_'+ arr[i]:'pop_density2010'+'_'+ arr[i]})
    return df

In [15]:
locations = world.groupby('location').first().reset_index() # The locations to query

In [16]:
locations.shape

(463, 7)

In [12]:
# Add static vars
locations_w_static = add_static_vars(locations.copy())

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.53s/it]


In [13]:
locations_w_static = locations_w_static.drop(columns=['Date', 'PM25', 'Lat', 'Long', 'city','country'], axis=1)
world_w_static = pd.merge(world, locations_w_static, on='location')

In [15]:
world_w_static.head()

,Date,location,city,country,PM25,Lat,Long,avg_vis_mean,cf_cvg_mean,avg_vis_min,cf_cvg_min,avg_vis_max,cf_cvg_max,pop_density2010_mean,pop_density2010_min,pop_density2010_max
0,2018-11-22,Camarones,DISTRITO FEDERAL,MX,34.095238,19.4684,-99.1697,577.686646,9.0,577.686646,9.0,577.686646,9.0,15686.94043,15686.94043,15686.94043
1,2018-11-23,Camarones,DISTRITO FEDERAL,MX,33.347826,19.4684,-99.1697,577.686646,9.0,577.686646,9.0,577.686646,9.0,15686.94043,15686.94043,15686.94043
2,2018-11-24,Camarones,DISTRITO FEDERAL,MX,41.625000,19.4684,-99.1697,577.686646,9.0,577.686646,9.0,577.686646,9.0,15686.94043,15686.94043,15686.94043
3,2018-11-25,Camarones,DISTRITO FEDERAL,MX,29.904762,19.4684,-99.1697,577.686646,9.0,577.686646,9.0,577.686646,9.0,15686.94043,15686.94043,15686.94043
4,2018-11-26,Camarones,DISTRITO FEDERAL,MX,30.833333,19.4684,-99.1697,577.686646,9.0,577.686646,9.0,577.686646,9.0,15686.94043,15686.94043,15686.94043


### Timeseries data (Sentinel-5P, weather data)

In [206]:
#timeseries data
# Image Collections
gfs = ee.ImageCollection("NOAA/GFS0P25") # Weather data

S5p_collections = {} # Sentinel 5p data, which comes in multiple collections
for COL in ['L3_NO2', 'L3_O3', 'L3_CO', 'L3_AER_AI']: # 
    S5p_collections[COL] = ee.ImageCollection('COPERNICUS/S5P/OFFL/'+COL).map(lambda image: image.addBands\
                                                                              (image.metadata('system:time_start')))
# Properties for each image we want to keep
s5p_props = {
    'L3_NO2':['NO2_column_number_density', 'tropospheric_NO2_column_number_density', 'stratospheric_NO2_column_number_density',\
              'NO2_slant_column_number_density', 'tropopause_pressure'],
    'L3_O3':['O3_column_number_density'],
    'L3_CO':['CO_column_number_density', 'H2O_column_number_density', 'cloud_height'],
    'L3_AER_AI':['absorbing_aerosol_index']
}

def add_timeseries(df, dates, reducer=ee.Reducer.first()):
    # Prepare dataframe with date x city
    date_col = []
    location_col = []
    for d in dates:
        for c in df.location.unique():
            date_col.append(d)
            location_col.append(c)

    data = pd.DataFrame({
        'Date':date_col,
        'location':location_col
    })
    data = pd.merge(data, df[['location', 'Lat', 'Long']], how='left', on='location')
    
    for d in tqdm(dates):
        # Weather is easy - a single image from the right date
        weather_image = gfs.filter(ee.Filter.date(str(d.date()), str((d+timedelta(days=1)).date()))).first() # Filter to get the relevant image
        # For the sentinel data, we get images from each collection and merge them
        s5p_images = []
        for COL in ['L3_NO2', 'L3_O3', 'L3_CO', 'L3_AER_AI']:
            collection = S5p_collections[COL].filter(ee.Filter.date(str((d-timedelta(days=5)).date()), str(d.date())))
            image = collection.qualityMosaic('system:time_start') # The most recent image
            image = image.select(s5p_props[COL])
            s5p_images.append(image)
        s5p_image = ee.ImageCollection(s5p_images).toBands() # Merge into one image
    
        # Sample the weather data
        samples = sample(weather_image, '', df['Lat'].values, df['Long'].values, reducer=reducer)
        for prop in samples[0].keys():
            data.loc[data.Date==d, prop] = [p[prop] for p in samples]
            
        # Sample the sentinel data
        samples = sample(s5p_image, '', df['Lat'].values, df['Long'].values)
        for prop in samples[0].keys():
            data.loc[data.Date==d, prop] = [p[prop] for p in samples]
            
    return data

In [ ]:
#test
dates = pd.date_range('2020-01-01', '2020-01-04', freq='1D')
add_timeseries(world.head(5).copy(), dates) # Show the result on the first 5 rows

In [17]:
world['Date'].min()

'2018-11-22'

In [18]:
world['Date'].max()

'2021-01-31'

In [19]:
world.shape

(96195, 7)

In [30]:
#weather data is until 2020-11-30
dates_2018 = pd.date_range('2018-11-22', '2018-12-31', freq='1D')
dates_2019 = pd.date_range('2019-01-01', '2019-12-31', freq='1D')
dates_2020 = pd.date_range('2020-01-01', '2020-12-31', freq='1D')
dates_2021 = pd.date_range('2021-01-01', '2021-01-31', freq='1D')

In [28]:
CO = pd.read_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/02_OpenAQ/Colombia.csv')
EC = pd.read_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/02_OpenAQ/Ecuador.csv')
PE = pd.read_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/02_OpenAQ/Peru.csv')
MX = pd.read_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/02_OpenAQ/Mexico.csv')

In [22]:
def trans_data(data):
    data = data.rename(columns={"date.utc": "Date", "coordinates.latitude": "Lat", 
                                 "coordinates.longitude":"Long", "value": "PM25"})
    data['Date'] = data['Date'].astype(str).str[:10]
    data = data.drop(columns=['unit', 'parameter'], axis =1)
    data = data[['location', 'Date', 'city', 'country', 'PM25', 'Lat','Long']]
    data['PM25'] = data.groupby(['Date','location'])['PM25'].transform(lambda x: x.mean())
    data = data.groupby(['Date', 'location']).first().reset_index()
    return data

In [25]:
CO = trans_data(CO)
EC = trans_data(EC)
PE = trans_data(PE)
MX = trans_data(MX)

In [29]:
print(CO.shape)
print(EC.shape)
print(PE.shape)
MX.shape

(200000, 9)
(4087, 9)
(42828, 9)


(331806, 9)

In [30]:
# Split the data because it doesn't run in GEE when it's too large
CO_1 = CO.iloc[:5000,:] 
CO_2 = CO.iloc[5001:8696,:]
MX_1 = MX.iloc[:4000,:] 
MX_2 = MX.iloc[4001:8000,:]
MX_3 = MX.iloc[8001:12000, :]
MX_4 = MX.iloc[12001:16374, :]
world_1 = world.iloc[:5000,:] 
world_2 = world.iloc[5001:10000,:]
world_3 = world.iloc[10001:15000, :]
world_4 = world.iloc[15001:20000, :]
world_5 = world.iloc[20001:25000, :]
world_6 = world.iloc[25001:30000, :]
world_7 = world.iloc[30001:35000, :]
world_8 = world.iloc[35001:40000,:]
world_9 = world.iloc[40001:45000, :]
world_10 = world.iloc[45001:50000, :]
world_11 = world.iloc[50001:55000, :]
world_12 = world.iloc[55001:60000, :]
world_13 = world.iloc[60001:62482, :]

In [43]:
# Get the timeseries data
ts_CO1 = add_timeseries(CO_1.copy(), dates_2018)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [12:14<00:00, 18.35s/it]


In [46]:
ts_CO2 = add_timeseries(CO_1.copy(), dates_2019)

100%|██████████████████████████████████████████████████████████████████████████████| 365/365 [1:30:21<00:00, 14.85s/it]


In [47]:
ts_CO3 = add_timeseries(CO_1.copy(), dates_2020)

100%|██████████████████████████████████████████████████████████████████████████████| 366/366 [1:22:48<00:00, 13.57s/it]


In [48]:
ts_CO4 = add_timeseries(CO_1.copy(), dates_2021)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [05:25<00:00, 10.51s/it]


In [49]:
ts_CO5 = add_timeseries(CO_2.copy(), dates_2018)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [06:18<00:00,  9.45s/it]


In [50]:
ts_CO6 = add_timeseries(CO_2.copy(), dates_2019)

100%|██████████████████████████████████████████████████████████████████████████████| 365/365 [1:04:02<00:00, 10.53s/it]


In [51]:
ts_CO7 = add_timeseries(CO_2.copy(), dates_2020)

100%|██████████████████████████████████████████████████████████████████████████████| 366/366 [1:01:52<00:00, 10.14s/it]


In [52]:
ts_CO8 = add_timeseries(CO_2.copy(), dates_2021)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [04:57<00:00,  9.61s/it]


In [53]:
ts_EC1 = add_timeseries(EC.copy(), dates_2018)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [01:23<00:00,  2.08s/it]


In [54]:
ts_EC2 = add_timeseries(EC.copy(), dates_2019)

100%|████████████████████████████████████████████████████████████████████████████████| 365/365 [12:52<00:00,  2.12s/it]


In [55]:
ts_EC3 = add_timeseries(EC.copy(), dates_2020)

100%|████████████████████████████████████████████████████████████████████████████████| 366/366 [12:39<00:00,  2.08s/it]


In [56]:
ts_EC4 = add_timeseries(EC.copy(), dates_2021)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [01:02<00:00,  2.02s/it]


In [57]:
ts_PE1 = add_timeseries(PE.copy(), dates_2018)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [03:24<00:00,  5.12s/it]


In [58]:
ts_PE2 = add_timeseries(PE.copy(), dates_2019)

100%|████████████████████████████████████████████████████████████████████████████████| 365/365 [34:30<00:00,  5.67s/it]


In [59]:
ts_PE3 = add_timeseries(PE.copy(), dates_2020)

100%|████████████████████████████████████████████████████████████████████████████████| 366/366 [33:20<00:00,  5.46s/it]


In [60]:
ts_PE4 = add_timeseries(PE.copy(), dates_2021)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [02:32<00:00,  4.92s/it]


In [ ]:
ts_MX1 = add_timeseries(MX_1.copy(), dates_2018)

In [ ]:
ts_MX2 = add_timeseries(MX_1.copy(), dates_2019)

In [ ]:
ts_MX3 = add_timeseries(MX_1.copy(), dates_2020)

In [ ]:
ts_MX4 = add_timeseries(MX_1.copy(), dates_2021)

In [ ]:
ts_MX5 = add_timeseries(MX_2.copy(), dates_2018)

In [ ]:
ts_MX6 = add_timeseries(MX_2.copy(), dates_2019)

In [ ]:
ts_MX7 = add_timeseries(MX_2.copy(), dates_2020)

In [ ]:
ts_MX8 = add_timeseries(MX_2.copy(), dates_2021)

In [ ]:
ts_MX9 = add_timeseries(MX_3.copy(), dates_2018)

In [ ]:
ts_MX10 = add_timeseries(MX_3.copy(), dates_2019)

In [ ]:
ts_MX11 = add_timeseries(MX_3.copy(), dates_2020)

In [ ]:
ts_MX12 = add_timeseries(MX_3.copy(), dates_2021)

In [ ]:
ts_MX13 = add_timeseries(MX_4.copy(), dates_2018)

In [ ]:
ts_MX14 = add_timeseries(MX_4.copy(), dates_2019)

In [ ]:
ts_MX15 = add_timeseries(MX_4.copy(), dates_2020)

In [ ]:
ts_MX16 = add_timeseries(MX_4.copy(), dates_2021)

In [31]:
world_1.shape

(5000, 7)

In [ ]:
ts1 = add_timeseries(world_1.copy(), dates_2018)

In [ ]:
ts2 = add_timeseries(world_1.copy(), dates_2019)

In [ ]:
ts3 = add_timeseries(world_1.copy(), dates_2020)

In [ ]:
ts4 = add_timeseries(world_1.copy(), dates_2021)

In [ ]:
ts5 = add_timeseries(world_2.copy(), dates_2018)

In [ ]:
ts6 = add_timeseries(world_2.copy(), dates_2019)

In [ ]:
ts7 = add_timeseries(world_2.copy(), dates_2020)

In [ ]:
ts8 = add_timeseries(world_2.copy(), dates_2021)

In [ ]:
ts9 = add_timeseries(world_3.copy(), dates_2018)

In [ ]:
ts10 = add_timeseries(world_3.copy(), dates_2019)

In [ ]:
ts11 = add_timeseries(world_3.copy(), dates_2020)

In [ ]:
ts12 = add_timeseries(world_3.copy(), dates_2021)

In [ ]:
ts13 = add_timeseries(world_4.copy(), dates_2018)

In [ ]:
ts14 = add_timeseries(world_4.copy(), dates_2019)

In [ ]:
ts15 = add_timeseries(world_4.copy(), dates_2020)

In [ ]:
ts16 = add_timeseries(world_4.copy(), dates_2021)

In [ ]:
ts17 = add_timeseries(world_5.copy(), dates_2018)

In [ ]:
ts18 = add_timeseries(world_5.copy(), dates_2019)

In [ ]:
ts19 = add_timeseries(world_5.copy(), dates_2020)

In [ ]:
ts20 = add_timeseries(world_5.copy(), dates_2021)

In [ ]:
ts21 = add_timeseries(world_6.copy(), dates_2018)

In [ ]:
ts22 = add_timeseries(world_6.copy(), dates_2019)

In [ ]:
ts23 = add_timeseries(world_6.copy(), dates_2020)

In [ ]:
ts24 = add_timeseries(world_6.copy(), dates_2021)

In [ ]:
ts25 = add_timeseries(world_7.copy(), dates_2018)

In [ ]:
ts26 = add_timeseries(world_7.copy(), dates_2019)

In [ ]:
ts27 = add_timeseries(world_7.copy(), dates_2020)

In [ ]:
ts28 = add_timeseries(world_7.copy(), dates_2021)

In [ ]:
ts29 = add_timeseries(world_8.copy(), dates_2018)

In [ ]:
ts30 = add_timeseries(world_8.copy(), dates_2019)

In [ ]:
ts31 = add_timeseries(world_8.copy(), dates_2020)

In [ ]:
ts32 = add_timeseries(world_8.copy(), dates_2021)

In [ ]:
ts33 = add_timeseries(world_9.copy(), dates_2018)

In [ ]:
ts34 = add_timeseries(world_9.copy(), dates_2019)

In [ ]:
ts35 = add_timeseries(world_9.copy(), dates_2020)

In [ ]:
ts36 = add_timeseries(world_9.copy(), dates_2021)

In [ ]:
ts37 = add_timeseries(world_10.copy(), dates_2018)

In [ ]:
ts38 = add_timeseries(world_10.copy(), dates_2019)

In [ ]:
ts39 = add_timeseries(world_10.copy(), dates_2020)

In [ ]:
ts40 = add_timeseries(world_10.copy(), dates_2021)

In [ ]:
ts41 = add_timeseries(world_11.copy(), dates_2018)

In [ ]:
ts42 = add_timeseries(world_11.copy(), dates_2019)

In [ ]:
ts43 = add_timeseries(world_11.copy(), dates_2020)

In [ ]:
ts44 = add_timeseries(world_11.copy(), dates_2021)

In [ ]:
ts46 = add_timeseries(world_12.copy(), dates_2019)

In [ ]:
ts47 = add_timeseries(world_12.copy(), dates_2020)

In [ ]:
ts48 = add_timeseries(world_12.copy(), dates_2021)

In [ ]:
ts49 = add_timeseries(world_13.copy(), dates_2018)

In [ ]:
ts50 = add_timeseries(world_13.copy(), dates_2019)

In [ ]:
ts51 = add_timeseries(world_13.copy(), dates_2020)

In [ ]:
ts52 = add_timeseries(world_13.copy(), dates_2021)

In [ ]:
PT1 = add_timeseries(PT_1.copy(), dates_2018)

In [ ]:
PT2 = add_timeseries(PT_1.copy(), dates_2019)

In [ ]:
PT3 = add_timeseries(PT_1.copy(), dates_2020)

In [ ]:
PT4 = add_timeseries(PT_1.copy(), dates_2021)

In [ ]:
PT5 = add_timeseries(PT_2.copy(), dates_2018)

In [ ]:
PT6 = add_timeseries(PT_2.copy(), dates_2019)

In [ ]:
PT7 = add_timeseries(PT_2.copy(), dates_2020)

In [ ]:
PT8 = add_timeseries(PT_2.copy(), dates_2021)

In [ ]:
PT8.to_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/PT8.csv', index=False)

### NDVI (vegetation index)

In [173]:
# Add vegetation index
def add_static_vars2(df, scale = 8000):
    vegs = ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5").filter(ee.Filter.date('2018-11-22', '2021-01-31')).first()
    veg_list = [vegs]

    for veg in tqdm(veg_list):
        for i, reducer in enumerate([ee.Reducer.mean(), ee.Reducer.min(), ee.Reducer.max()]):
            sampled_values = sample(veg, '', df['Lat'].values, df['Long'].values, reducer=reducer, scale=scale)
            for k in sampled_values[0].keys():
                arr = ['mean', 'min', 'max']
                df[k+'_'+str(scale)+'_' + arr[i]] = [sv[k] if k in sv.keys() else None for sv in sampled_values]
                if k == arr[i]:
                    df = df.rename(columns={k+'_'+str(scale)+'_' + arr[i]:'veg_index'+'_'+str(scale)+'_' + arr[i]})
    return df

In [ ]:
#test
#dates = pd.date_range('2020-01-01', '2020-01-04', freq='1D')
add_static_vars2(locations.head(5).copy()) # Show the result on the first 5 rows

In [18]:
# Add static vars
locations_w_veg = add_static_vars2(locations.copy())

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.02s/it]


In [19]:
locations_w_veg = locations_w_veg.drop(columns=['Date', 'PM25', 'Lat', 'Long', 'city','country'], axis=1)
world_w_veg = pd.merge(world, locations_w_veg, on='location')

In [21]:
world_w_veg.isna().sum()

Date                       0
location                   0
city                    4163
country                    0
PM25                       0
Lat                        0
Long                       0
NDVI_8500_mean         10013
QA_8500_mean               0
TIMEOFDAY_8500_mean     5983
NDVI_8500_min          14139
QA_8500_min                0
TIMEOFDAY_8500_min      8332
NDVI_8500_max          14139
QA_8500_max                0
TIMEOFDAY_8500_max      8332
dtype: int64

### Elevation data

In [247]:
def add_static_vars3(df, scale = 2000):
    elevations = ee.Image('USGS/GMTED2010').select('be75')
    elevation_list = [elevations]

    for elevation in tqdm(elevation_list):
        for i, reducer in enumerate([ee.Reducer.mean(), ee.Reducer.min(), ee.Reducer.max()]):
            sampled_values = sample(elevation, '', df['Lat'].values, df['Long'].values, reducer=reducer, scale=scale)
            for k in sampled_values[0].keys():
                arr = ['mean', 'min', 'max']
                df[k+'_'+str(scale)+'_' + arr[i]] = [sv[k] if k in sv.keys() else None for sv in sampled_values]
                if k == arr[i]:
                    df = df.rename(columns={k+'_'+str(scale)+'_' + arr[i]:'elevation'+'_'+str(scale)+'_' + arr[i]})
    return df

In [24]:
locations_w_elevation = add_static_vars3(locations.copy())

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.50s/it]


In [25]:
locations_w_elevation = locations_w_elevation.drop(columns=['Date', 'PM25', 'Lat', 'Long', 'city','country'], axis=1)
world_w_elevation = pd.merge(world, locations_w_elevation, on='location')

In [27]:
world_w_elevation.isna().sum()

Date                     0
location                 0
city                  4163
country                  0
PM25                     0
Lat                      0
Long                     0
elevation_500_mean      66
elevation_500_min       66
elevation_500_max       66
dtype: int64

### Human modification index

In [64]:
final3 = pd.read_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/final3.csv')

In [65]:
final3_locations = final3.groupby('location').first().reset_index() # The locations to query

In [99]:
image = ee.ImageCollection("CSP/HM/GlobalHumanModification").first()
test = sample(image, '', [-18.136], [30.15], scale=1000) # See image value for a single location
test

[{'first': 0.6889050602912903}]

In [340]:
# CSP gHM: Global Human Modification
def add_static_vars4(df, scale = 100):
    lands = ee.ImageCollection("CSP/HM/GlobalHumanModification").first()
    land_list = [lands]

    for land in tqdm(land_list):
        for i, reducer in enumerate([ee.Reducer.mean(), ee.Reducer.min(), ee.Reducer.max()]):
            sampled_values = sample(land, '', df['Lat'].values, df['Long'].values, reducer=reducer, scale=scale)
            for k in sampled_values[0].keys():
                arr = ['mean', 'min', 'max']
                df[k+'_'+str(scale)+'_' + arr[i]] = [sv[k] if k in sv.keys() else None for sv in sampled_values]
                if k == arr[i]:
                    df = df.rename(columns={k+'_'+str(scale)+'_' + arr[i]:'Human_modification'+'_'+str(scale)+'_' + arr[i]})
    return df

In [120]:
final3_w_add_static = add_static_vars4(final3_locations, scale = 500)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.06s/it]


In [121]:
final3_w_add_static.loc[final3_w_add_static['Human_modification_500_mean'].isnull()]

,location,Date,location_ID,Lat_x,Long_x,Precipitable_water,RH,SH,Temp,U_wind,...,NDVI_8500_mean,QA_8500_mean,elevation_500_mean,PM25,mean_100_mean,mean_200_mean,Human_modification_500_mean,mean_1000_mean,Human_modification_500_min,Human_modification_500_max
37,"Borivali East, Mumbai - MPCB",2020-04-05,38,19.132760,72.520480,21.600000,74.900002,0.01667,27.051385,0.533428,...,NaN,138.0,0.0,20.996818,NaN,NaN,NaN,NaN,NaN,NaN
73,Ciudad Bolivar,2020-10-29,74,-74.166278,4.577806,28.898485,93.916666,0.01250,14.984855,0.008136,...,-146.826949,-32638.0,NaN,21.620220,NaN,NaN,NaN,NaN,NaN,NaN
179,"Khindipada-Bhandup West, Mumbai - IITM",2020-12-16,180,19.099180,72.553390,30.800001,67.400002,0.01380,25.694421,-2.181769,...,NaN,138.0,0.0,867.213000,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
final3.head(1)

,Date,location,location_ID,Lat_x,Long_x,Precipitable_water,RH,SH,Temp,U_wind,...,H2O,Cloud,Aerosol,avg_vis_mean,cf_cvg_mean,pop_density2010_mean,NDVI_8500_mean,QA_8500_mean,elevation_500_mean,PM25
0,2018-11-22,Camarones,53,19.4684,-99.1697,14.347721,39.937691,0.006379,18.561679,0.085962,...,725.669616,558.024486,-0.609685,577.686646,9.0,15686.94043,599.949881,24101.77088,2244.655991,34.095238


In [123]:
final4 = pd.merge(left =final3, right = final3_w_add_static, on = 'location_ID', how='left')

In [125]:
final4.columns

Index(['Date_x', 'location_x', 'location_ID', 'Lat_x_x', 'Long_x_x',
       'Precipitable_water_x', 'RH_x', 'SH_x', 'Temp_x', 'U_wind_x',
       'V_wind_x', 'NO2_column_x', 'NO2_slant_x', 'NO2_strato_x', 'Pressure_x',
       'NO2_trop_x', 'O3_x', 'CO_x', 'H2O_x', 'Cloud_x', 'Aerosol_x',
       'avg_vis_mean_x', 'cf_cvg_mean_x', 'pop_density2010_mean_x',
       'NDVI_8500_mean_x', 'QA_8500_mean_x', 'elevation_500_mean_x', 'PM25_x',
       'location_y', 'Date_y', 'Lat_x_y', 'Long_x_y', 'Precipitable_water_y',
       'RH_y', 'SH_y', 'Temp_y', 'U_wind_y', 'V_wind_y', 'NO2_column_y',
       'NO2_slant_y', 'NO2_strato_y', 'Pressure_y', 'NO2_trop_y', 'O3_y',
       'CO_y', 'H2O_y', 'Cloud_y', 'Aerosol_y', 'avg_vis_mean_y',
       'cf_cvg_mean_y', 'pop_density2010_mean_y', 'NDVI_8500_mean_y',
       'QA_8500_mean_y', 'elevation_500_mean_y', 'PM25_y', 'mean_100_mean',
       'mean_200_mean', 'Human_modification_500_mean', 'mean_1000_mean',
       'Human_modification_500_min', 'Human_modificatio

In [126]:
final5 = final4.copy().drop(columns = ['Lat_x_x', 'Long_x_x','location_y', 'Date_y', 'Lat_x_y', 'Long_x_y', 
                                       'Precipitable_water_y',
       'RH_y', 'SH_y', 'Temp_y', 'U_wind_y', 'V_wind_y', 'NO2_column_y',
       'NO2_slant_y', 'NO2_strato_y', 'Pressure_y', 'NO2_trop_y', 'O3_y',
       'CO_y', 'H2O_y', 'Cloud_y', 'Aerosol_y', 'avg_vis_mean_y',
       'cf_cvg_mean_y', 'pop_density2010_mean_y', 'NDVI_8500_mean_y',
       'QA_8500_mean_y', 'elevation_500_mean_y', 'PM25_y', 'mean_100_mean',
       'mean_200_mean', 'mean_1000_mean',
       'Human_modification_500_min', 'Human_modification_500_max'], axis=1)

In [128]:
final5.to_csv('C:/Users/ayako/Documents/Berkeley Public Health/MPH Capstone_Air pollution and ARI/05_PM2.5 Prediction/final5.csv', index=False)